# Demo

let's analyze few homes



### step 1: who might be eligible 

Santa Clara County 车库改造 + detached adu

Alameda County - 车库改造 + detached adu

la county - 车库改造最多

San Francisco County - 车库改造居多

### find price per sqft high and total price not high
# lot size / home size; 
total price / price per sqft





In [16]:
# Read JSON data from county files
import json
import pandas as pd
import os

#display width change to 1000
pd.set_option('display.max_colwidth', 20)



In [17]:
# Define file paths
data_dir = "../data/raw/"
county_files = {
    "Santa Clara": "20250822_114114_santa_clara_county_ca_santa_clara_county.json",
    "Alameda": "20250822_114052_alameda_county_ca_alameda_county.json", 
    "Los Angeles": "20250822_113940_los_angeles_county_ca_los_angeles_county.json"
}

# Load data from each county
county_data = {}
for county_name, filename in county_files.items():
    filepath = os.path.join(data_dir, filename)
    with open(filepath, 'r') as f:
        data = json.load(f)
        county_data[county_name] = data
        print(f"Loaded {county_name} County: {len(data['listings'])} listings")

# Convert listings to DataFrames for easier analysis
county_dfs = {}
for county_name, data in county_data.items():
    # Extract relevant fields from each listing
    listings = []
    for listing in data['listings']:
        listing_data = {
            'address': listing.get('location', {}).get('address', {}).get('line', ''),
            'city': listing.get('location', {}).get('address', {}).get('city', ''),
            'zip': listing.get('location', {}).get('address', {}).get('postal_code', ''),
            'price': listing.get('list_price', 0),
            'price_per_sqft': listing.get('price_per_sqft', 0),
            'sqft': listing.get('description', {}).get('sqft', 0),
            'lot_sqft': listing.get('description', {}).get('lot_sqft', 0),
            'beds': listing.get('description', {}).get('beds', 0),
            'baths': listing.get('description', {}).get('baths', 0),
            'description': listing.get('description', {}).get('text', ''),
            'year_built': listing.get('description', {}).get('year_built', 0),
            'garage': listing.get('description', {}).get('garage', 0),
            'property_type': listing.get('description', {}).get('type', ''),
            'listing_id': listing.get('listing_id', ''),
            'longitude': listing.get('location', {}).get('address', {}).get('coordinate', {}).get('lon', 0),
            'latitude': listing.get('location', {}).get('address', {}).get('coordinate', {}).get('lat', 0),
            'url': listing.get('href', '')
        }
        listings.append(listing_data)
    
    county_dfs[county_name] = pd.DataFrame(listings)
    print(f"{county_name} DataFrame created with shape: {county_dfs[county_name].shape}")

# Display basic stats for each county
for county_name, df in county_dfs.items():
    print(f"\n{county_name} County Statistics:")
    print(f"  Average price: ${df['price'].mean():,.0f}")
    print(f"  Average price/sqft: ${df['price_per_sqft'].mean():,.0f}")
    print(f"  Average sqft: {df['sqft'].mean():,.0f}")
    print(f"  Average lot sqft: {df['lot_sqft'].mean():,.0f}")

#conver to pandas dataframe
county_dfs = pd.concat(county_dfs.values())

#display
county_dfs.head(5)


Loaded Santa Clara County: 1366 listings
Loaded Alameda County: 1841 listings
Loaded Los Angeles County: 4513 listings
Santa Clara DataFrame created with shape: (1366, 17)
Alameda DataFrame created with shape: (1841, 17)
Los Angeles DataFrame created with shape: (4513, 17)

Santa Clara County Statistics:
  Average price: $2,192,131
  Average price/sqft: $0
  Average sqft: 2,104
  Average lot sqft: 28,001

Alameda County Statistics:
  Average price: $1,557,264
  Average price/sqft: $0
  Average sqft: 2,202
  Average lot sqft: 229,482

Los Angeles County Statistics:
  Average price: $1,346,727
  Average price/sqft: $0
  Average sqft: 2,073
  Average lot sqft: 465,014


,address,city,zip,price,price_per_sqft,sqft,lot_sqft,beds,baths,description,year_built,garage,property_type,listing_id,longitude,latitude,url
0,5939 Pala Mesa Dr,San Jose,95123,1899888,0,2451.0,4715.0,4,3.0,Welcome to 5939 ...,2000.0,0,single_family,2985606552,-121.813919,37.240225,https://www.real...
1,119 Belcrest Dr,Los Gatos,95032,2699000,0,1748.0,10376.0,4,2.0,Beautifully reno...,1965.0,0,single_family,2985604175,-121.914295,37.230329,https://www.real...
2,2055 Rucker Ave,Gilroy,95020,1649777,0,1172.0,306662.0,3,2.0,Brand-new home o...,2025.0,0,single_family,2985604058,-121.556420,37.062856,https://www.real...
3,5850 Chesbro Ave,San Jose,95123,1499900,0,1777.0,6000.0,3,3.0,Welcome to 5850 ...,1964.0,0,single_family,2985597086,-121.843059,37.244173,https://www.real...
4,1641 Rue Avati,San Jose,95131,1349888,0,1205.0,6247.0,3,2.0,Welcome to 1641 ...,1973.0,0,single_family,2985596768,-121.880692,37.388961,https://www.real...


In [18]:
#calculate price per sqft / total price
county_dfs['price_per_sqft'] = county_dfs['price'] / county_dfs['sqft']
print(county_dfs.shape)

#analyze county_dfs['price_per_sqft'] 
#plot price per sqft
# import matplotlib.pyplot as plt
# plt.figure(figsize=(10, 6))
# plt.hist(county_dfs['price_per_sqft'], bins=50, color='skyblue', edgecolor='black')
# plt.title('Distribution of Price per Square Foot')
# plt.xlabel('Price per Square Foot')
# plt.ylabel('Frequency')
# plt.show()

#filter price and unit price
county_dfs = county_dfs[(county_dfs['price']<3000000)&(county_dfs['price']>700000)]
print(county_dfs.shape)

county_dfs = county_dfs[(county_dfs['price_per_sqft']>600)&(county_dfs['price_per_sqft']<1200)]
print(county_dfs.shape)
#calculate  lot sqft/home sqft ratio
county_dfs['lot_sqft_ratio'] = county_dfs['lot_sqft'] / county_dfs['sqft']
print(county_dfs.shape)

#remove those who have lot_sqft_ratio > 100
county_dfs = county_dfs[(county_dfs['lot_sqft_ratio']<20)&(county_dfs['lot_sqft_ratio']>3)]
print(county_dfs.shape)
#select lot sqft > 5000 and home sqft < 2500
county_dfs = county_dfs[(county_dfs['lot_sqft']>5000)&(county_dfs['sqft']<2500)]
print(county_dfs.shape)
#select home year built < 2006
county_dfs = county_dfs[county_dfs['year_built']<2010]
print(county_dfs.shape)


#analyze lot_sqft_ratio
#plot lot_sqft_ratio
# import matplotlib.pyplot as plt
# plt.figure(figsize=(10, 6))
# plt.hist(county_dfs['lot_sqft_ratio'], bins=50, color='skyblue', edgecolor='black')
# plt.title('Distribution of Lot Square Footage Ratio')
# plt.xlabel('Lot Square Footage Ratio')
# plt.ylabel('Frequency')
# plt.show()

county_dfs = county_dfs.reset_index(drop=True)
county_dfs


#remove those who have adu already

(7720, 17)
(7362, 17)
(4410, 17)
(4410, 18)
(3205, 18)
(2589, 18)
(2563, 18)


,address,city,zip,price,price_per_sqft,sqft,lot_sqft,beds,baths,description,year_built,garage,property_type,listing_id,longitude,latitude,url,lot_sqft_ratio
0,5850 Chesbro Ave,San Jose,95123,1499900,844.063028,1777.0,6000.0,3,3.0,Welcome to 5850 ...,1964.0,0,single_family,2985597086,-121.843059,37.244173,https://www.real...,3.376477
1,1641 Rue Avati,San Jose,95131,1349888,1120.239004,1205.0,6247.0,3,2.0,Welcome to 1641 ...,1973.0,0,single_family,2985596768,-121.880692,37.388961,https://www.real...,5.184232
2,5991 Sorrel Ave,San Jose,95123,1600000,819.672131,1952.0,6120.0,5,2.0,Welcome to 5991 ...,1965.0,0,single_family,2985592836,-121.825358,37.239649,https://www.real...,3.135246
3,266 Vista Ave,San Jose,95127,900000,813.008130,1107.0,7000.0,2,1.0,PLEASE DO NOT DI...,1920.0,0,single_family,2985590778,-121.825939,37.380543,https://www.real...,6.323397
4,2936 Stutz Way,San Jose,95148,1650000,1057.692308,1560.0,6000.0,4,2.0,Welcome to 2936 ...,1976.0,0,single_family,2985588295,-121.793813,37.321354,https://www.real...,3.846154
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2558,525 Hacienda Dr,Monrovia,91016,1125000,1057.330827,1064.0,7410.0,2,1.0,Welcome to this ...,1949.0,0,single_family,2984992140,-117.979528,34.145859,https://www.real...,6.964286
2559,6460 W 84th St,Los Angeles,90045,1595000,1126.412429,1416.0,6105.0,3,2.0,Do not miss this...,1944.0,0,single_family,2984216377,-118.401232,33.962698,https://www.real...,4.311441
2560,4040 Lehman Rd,La Crescenta,91214,1275000,728.987993,1749.0,8125.0,3,2.0,Great Opportunit...,1956.0,0,single_family,2984221760,-118.264551,34.231923,https://www.real...,4.645512
2561,2435 Old Grand St,Santa Ana,92705,950000,611.719253,1553.0,8600.0,3,2.0,Located in the h...,1955.0,0,single_family,2984997579,-117.849471,33.770116,https://www.real...,5.537669


In [19]:
import re
# 定义ADU相关的关键词模式（不区分大小写）
adu_pattern = re.compile(r'\b(adu|accessory\s+dwelling|granny\s+unit|in-?law\s+unit|secondary\s+unit)\b', re.IGNORECASE)
#add True or False to county_dfs and new column name is existing_adu (1 if adu_pattern is found in description, 0 otherwise)
#handle null values
county_dfs['description'] = county_dfs['description'].fillna(' ')
county_dfs['existing_adu'] = county_dfs['description'].apply(lambda x: 1 if adu_pattern.search(x) is not None else 0)

#print count of existing_adu 1 and 0
print(county_dfs[county_dfs['existing_adu']==0].shape)
print(county_dfs[county_dfs['existing_adu']==1].shape)

#display county_dfs
county_dfs[county_dfs['existing_adu']==1]


(2002, 19)
(561, 19)


,address,city,zip,price,price_per_sqft,sqft,lot_sqft,beds,baths,description,year_built,garage,property_type,listing_id,longitude,latitude,url,lot_sqft_ratio,existing_adu
7,818 N 13th St,San Jose,95112,1270000,965.045593,1316.0,5375.0,3,1.0,Charming 3-Bedro...,1910.0,0,single_family,2985583896,-121.889444,37.357846,https://www.real...,4.084347,1
16,771 N 16th St,San Jose,95112,999900,824.319868,1213.0,6625.0,3,2.0,Exquisitely remo...,1947.0,0,single_family,2985568233,-121.886224,37.357566,https://www.real...,5.461665,1
22,3305 Hickerson Dr,San Jose,95127,999900,691.015895,1447.0,5225.0,3,3.0,Charming Home wi...,1962.0,0,single_family,2985545067,-121.811701,37.357361,https://www.real...,3.610919,1
27,82 Hollywood Ave,San Jose,95112,1288000,658.486708,1956.0,5922.0,4,2.0,"Offering $10, 00...",1920.0,0,single_family,2985536266,-121.875057,37.318495,https://www.real...,3.027607,1
39,336 West Ct,San Jose,95116,988000,1020.661157,968.0,5625.0,4,3.0,"Rare, flexible s...",1927.0,0,single_family,2985510089,-121.868025,37.354426,https://www.real...,5.810950,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2549,867 New York Dr,Altadena,91001,1400000,664.767331,2106.0,7314.0,4,3.0,Welcome to 867 N...,1937.0,0,single_family,2984232934,-118.132487,34.178956,https://www.real...,3.472934,1
2554,13345 Burbank Blvd,Van Nuys,91401,1100000,947.459087,1161.0,6719.0,3,1.0,SOLD Before PROC...,1948.0,0,single_family,2985005789,-118.424131,34.172431,https://www.real...,5.787252,1
2555,4803 Gambier St,Los Angeles,90032,729000,767.368421,950.0,7362.0,3,2.0,Think owning a h...,1923.0,0,single_family,2985005429,-118.179680,34.081222,https://www.real...,7.749474,1
2557,557 Black Hills Dr,Claremont,91711,900000,604.838710,1488.0,9875.0,4,2.0,Welcome to a hom...,1962.0,0,single_family,2985004154,-117.705783,34.115812,https://www.real...,6.636425,1


In [20]:
county_dfs_sample = county_dfs.sample(500)
county_dfs_sample = county_dfs_sample.reset_index(drop=True)
county_dfs_sample

,address,city,zip,price,price_per_sqft,sqft,lot_sqft,beds,baths,description,year_built,garage,property_type,listing_id,longitude,latitude,url,lot_sqft_ratio,existing_adu
0,15227 Graystone Ave,Norwalk,90650,869000,758.951965,1145.0,5068.0,3,2.0,This beautifully...,1949.0,0,single_family,2985414216,-118.093702,33.892176,https://www.real...,4.426201,1
1,1303 N Merona St,Anaheim,92805,898000,702.660407,1278.0,7039.0,3,2.0,"Modern Comfort, ...",1954.0,0,single_family,2984966173,-117.904341,33.851798,https://www.real...,5.507825,1
2,22849 Leadwell St,West Hills,91307,830000,638.953041,1299.0,7499.0,3,2.0,Welcome to this ...,1960.0,0,single_family,2984966073,-118.625318,34.203653,https://www.real...,5.772902,0
3,11373 Rampart Dr,Dublin,94568,1590000,702.917772,2262.0,13328.0,4,3.0,This stunning ex...,1965.0,0,single_family,2984939876,-121.948393,37.699774,https://www.real...,5.892131,0
4,5841 Arroyo Dr,Los Angeles,90042,1148000,911.111111,1260.0,5564.0,3,3.0,"Back Yard, Front...",1947.0,0,single_family,2985034060,-118.188427,34.106342,https://www.real...,4.415873,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,3109 Robinette Ave,Baldwin Park,91706,855000,682.907348,1252.0,5125.0,3,2.0,Welcome to this ...,1956.0,0,single_family,2984693647,-117.986066,34.068558,https://www.real...,4.093450,1
496,24905 Skyland Rd,Los Gatos,95033,1050000,637.135922,1648.0,27181.0,3,2.0,First Time on th...,1980.0,0,single_family,2984082785,-121.928855,37.110571,https://www.real...,16.493325,0
497,14621 Chalet Ln,Huntington Beach,92647,1439000,761.375661,1890.0,6000.0,4,2.0,Modern Luxury Li...,1964.0,0,single_family,2984920295,-118.020943,33.749986,https://www.real...,3.174603,0
498,1925 Vallecito Dr,San Pedro,90732,1280000,981.595092,1304.0,7321.0,3,2.0,Panoramic ocean ...,1959.0,0,single_family,2985241278,-118.319606,33.726348,https://www.real...,5.614264,0


### Step 2： searching 管辖权 / zoning 

具体实施步骤：

  1. 转换 GPKG 文件坐标系

  # 读取 GPKG 文件
  gdf = gpd.read_file(file_path)

  # 从 EPSG:3857 转换为 EPSG:4326
  gdf_wgs84 = gdf.to_crs('EPSG:4326')

  2. 创建空间点对象

  # 从 county_dfs 创建 GeoDataFrame
  geometry = [Point(xy) for xy in zip(county_dfs.longitude, county_dfs.latitude)]
  properties_gdf = gpd.GeoDataFrame(county_dfs,
                                   crs='EPSG:4326',
                                   geometry=geometry)

  3. 执行空间连接查询

  # 空间连接：找出每个房产所在的 zoning 区域
  result = gpd.sjoin(properties_gdf, gdf_wgs84,
                     how='left',
                     predicate='within')


find zoning / 是否属于city建制区域等内容 其他内容，使用county_dfs_sample这个50rows 数据

In [21]:
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
from shapely.geometry import Point
import warnings
warnings.filterwarnings('ignore')

print("Libraries imported successfully")

Libraries imported successfully


In [22]:
# Step 1: Load and convert GPKG zoning data
print("Loading zoning data from GPKG file...")
file_path_north = "../data/raw/zoning-data/ca-north/California_Statewide_Zoning_North_4449911999654225179.gpkg"
file_path_south = "../data/raw/zoning-data/ca-south/California_Statewide_Zoning_South_685864265303796864.gpkg"

# Read GPKG file
gdf_zoning_north = gpd.read_file(file_path_north)
gdf_zoning_south = gpd.read_file(file_path_south)

gdf_zoning = pd.concat([gdf_zoning_north, gdf_zoning_south])

print(f"Loaded {len(gdf_zoning_north)} zoning polygons")
print(f"Original CRS: {gdf_zoning_north.crs}")


print(f"Loaded {len(gdf_zoning_south)} zoning polygons")
print(f"Original CRS: {gdf_zoning_south.crs}")


# Convert from EPSG:3857 to EPSG:4326
print("\nConverting from EPSG:3857 to EPSG:4326...")
gdf_zoning_wgs84 = gdf_zoning.to_crs('EPSG:4326')
print(f"Converted CRS: {gdf_zoning_wgs84.crs}")

# Display sample of zoning data
print("\nSample of zoning data columns:")
print(gdf_zoning_wgs84.columns.tolist())
print("\nFirst 3 records:")
gdf_zoning_wgs84[['County', 'Jurisdiction', 'Code', 'Description', 'New_Class']].head(3)

Loading zoning data from GPKG file...
Loaded 264417 zoning polygons
Original CRS: EPSG:3857
Loaded 304324 zoning polygons
Original CRS: EPSG:3857

Converting from EPSG:3857 to EPSG:4326...
Converted CRS: EPSG:4326

Sample of zoning data columns:
['County', 'Jurisdiction', 'Code', 'Description', 'Classkey', 'ucd_number', 'ucd_description', 'Source', 'Date', 'Version', 'New_Class', 'geometry']

First 3 records:


,County,Jurisdiction,Code,Description,New_Class
0,ALA,ALA,A,Agriculture,Agricultural
1,ALA,ALA,A,Agriculture,Agricultural
2,ALA,ALA,A,Agriculture,Agricultural


In [23]:
# Step 2: Create GeoDataFrame from county_dfs_sample
print("Creating GeoDataFrame from sample properties...")
print(f"Sample size: {len(county_dfs_sample)} properties")

# Create Point geometries from longitude and latitude
geometry = [Point(xy) for xy in zip(county_dfs_sample.longitude, county_dfs_sample.latitude)]

# Create GeoDataFrame
properties_gdf = gpd.GeoDataFrame(
    county_dfs_sample, 
    crs='EPSG:4326',  # WGS84 coordinate system
    geometry=geometry
)

print(f"GeoDataFrame created with CRS: {properties_gdf.crs}")
print(f"Total properties with valid coordinates: {len(properties_gdf)}")

# Display sample
print("\nSample properties:")
properties_gdf[['address', 'city', 'longitude', 'latitude', 'geometry']].head()

Creating GeoDataFrame from sample properties...
Sample size: 500 properties
GeoDataFrame created with CRS: EPSG:4326
Total properties with valid coordinates: 500

Sample properties:


,address,city,longitude,latitude,geometry
0,15227 Graystone Ave,Norwalk,-118.093702,33.892176,POINT (-118.0937...
1,1303 N Merona St,Anaheim,-117.904341,33.851798,POINT (-117.9043...
2,22849 Leadwell St,West Hills,-118.625318,34.203653,POINT (-118.6253...
3,11373 Rampart Dr,Dublin,-121.948393,37.699774,POINT (-121.9483...
4,5841 Arroyo Dr,Los Angeles,-118.188427,34.106342,POINT (-118.1884...


In [24]:
# Step 3: Perform spatial join to find zoning information
print("Performing spatial join to match properties with zoning...")
print("This may take a moment...")

# Create spatial index for faster queries
print("Creating spatial index for zoning data...")
gdf_zoning_wgs84.sindex

# Perform spatial join
# 'within' predicate finds which zoning polygon contains each property point
result_gdf = gpd.sjoin(
    properties_gdf, 
    gdf_zoning_wgs84, 
    how='left',  # Keep all properties even if no zoning match
    predicate='within'  # Check if property point is within zoning polygon
)

# Count matches
matched = result_gdf['index_right'].notna().sum()
total = len(result_gdf)
print(f"\nSpatial join complete!")
print(f"Properties with zoning match: {matched}/{total} ({matched/total*100:.1f}%)")
print(f"Properties without zoning: {total - matched}")

# Display columns
print("\nAvailable columns after join:")
print(result_gdf.columns.tolist())

Performing spatial join to match properties with zoning...
This may take a moment...
Creating spatial index for zoning data...

Spatial join complete!
Properties with zoning match: 501/501 (100.0%)
Properties without zoning: 0

Available columns after join:
['address', 'city', 'zip', 'price', 'price_per_sqft', 'sqft', 'lot_sqft', 'beds', 'baths', 'description', 'year_built', 'garage', 'property_type', 'listing_id', 'longitude', 'latitude', 'url', 'lot_sqft_ratio', 'existing_adu', 'geometry', 'index_right', 'County', 'Jurisdiction', 'Code', 'Description', 'Classkey', 'ucd_number', 'ucd_description', 'Source', 'Date', 'Version', 'New_Class']


In [25]:
# Step 4: Analyze zoning results
print("="*70)
print("ZONING ANALYSIS RESULTS")
print("="*70)

# Select relevant columns for display
zoning_columns = ['address', 'city', 'price', 'sqft', 'lot_sqft', 
                  'County', 'Jurisdiction', 'Code', 'Description', 'New_Class']

# Filter to show only matched properties
matched_properties = result_gdf[result_gdf['index_right'].notna()][zoning_columns]

print(f"\nTotal matched properties: {len(matched_properties)}")

# Analyze zoning types
if len(matched_properties) > 0:
    print("\n--- Zoning Type Distribution ---")
    zoning_dist = matched_properties['Description'].value_counts()
    print(zoning_dist.head(10))
    
    print("\n--- Zoning Class Distribution ---")
    class_dist = matched_properties['New_Class'].value_counts()
    print(class_dist.head(10))
    
    print("\n--- Jurisdiction Distribution ---")
    jurisdiction_dist = matched_properties['Jurisdiction'].value_counts()
    print(jurisdiction_dist.head(10))
    
    print("\n--- Sample Matched Properties ---")
    display_cols = ['address', 'city', 'Description', 'New_Class', 'Jurisdiction']
    print(matched_properties[display_cols].head(10))
else:
    print("No properties matched with zoning data")

ZONING ANALYSIS RESULTS

Total matched properties: 501

--- Zoning Type Distribution ---
Description
RESIDENTIAL                  97
R-1-8                        50
Single Family Residential    27
Single-Family Residence      26
                             22
R-1                          21
Residential                  13
R1                           13
RS-2                         12
Single-Family Residential    11
Name: count, dtype: int64

--- Zoning Class Distribution ---
New_Class
Suburban residential    354
Urban residential        97
Multi-family             24
Planned area             17
Exurban residential       3
Rural residential         2
Other                     2
Agricultural              1
Mobile home               1
Name: count, dtype: int64

--- Jurisdiction Distribution ---
Jurisdiction
Los Angeles    99
San Jose       66
LAN            22
Livermore      18
Pleasanton     14
Fremont        14
Long Beach     12
Anaheim        12
Burbank         9
Hayward         9
Na

In [26]:
# Step 5: Check jurisdiction status (incorporated vs unincorporated areas)
print("\n" + "="*70)
print("JURISDICTION ANALYSIS (City vs County)")
print("="*70)

# Analyze if properties are in incorporated cities or unincorporated county areas
if len(matched_properties) > 0:
    # Properties with specific city jurisdiction (incorporated)
    city_jurisdiction = matched_properties[
        ~matched_properties['Jurisdiction'].str.contains('County|COUNTY', case=False, na=False) &
        matched_properties['Jurisdiction'].notna()
    ]
    
    # Properties in county jurisdiction (unincorporated)
    county_jurisdiction = matched_properties[
        matched_properties['Jurisdiction'].str.contains('County|COUNTY', case=False, na=False)
    ]
    
    print(f"\nIncorporated (City) areas: {len(city_jurisdiction)} properties")
    print(f"Unincorporated (County) areas: {len(county_jurisdiction)} properties")
    
    if len(city_jurisdiction) > 0:
        print("\n--- Cities with Properties ---")
        print(city_jurisdiction['Jurisdiction'].value_counts().head())
    
    if len(county_jurisdiction) > 0:
        print("\n--- County Jurisdictions ---")
        print(county_jurisdiction['Jurisdiction'].value_counts().head())
    
    # Show sample properties from each category
    if len(city_jurisdiction) > 0:
        print("\n--- Sample City Properties ---")
        print(city_jurisdiction[['address', 'city', 'Jurisdiction', 'Description']].head(3))
    
    if len(county_jurisdiction) > 0:
        print("\n--- Sample County (Unincorporated) Properties ---")
        print(county_jurisdiction[['address', 'city', 'Jurisdiction', 'Description']].head(3))


JURISDICTION ANALYSIS (City vs County)

Incorporated (City) areas: 501 properties
Unincorporated (County) areas: 0 properties

--- Cities with Properties ---
Jurisdiction
Los Angeles    99
San Jose       66
LAN            22
Livermore      18
Pleasanton     14
Name: count, dtype: int64

--- Sample City Properties ---
               address        city Jurisdiction          Description
0  15227 Graystone Ave     Norwalk      Norwalk  Single-Family Re...
1     1303 N Merona St     Anaheim      Anaheim                 RS-2
2    22849 Leadwell St  West Hills  Los Angeles          RESIDENTIAL


In [27]:
# Step 6: Create summary table with all zoning information
print("\n" + "="*70)
print("COMPLETE ZONING SUMMARY TABLE")
print("="*70)

# Create a comprehensive summary DataFrame
if 'index_right' in result_gdf.columns:
    summary_df = result_gdf[['address', 'city', 'price', 'sqft', 'lot_sqft', 'lot_sqft_ratio',
                             'existing_adu', 'County', 'Jurisdiction', 'Code', 
                             'Description', 'New_Class']].copy()
    
    # Add zoning status
    summary_df['has_zoning'] = result_gdf['index_right'].notna()
    
    # Add jurisdiction type
    summary_df['jurisdiction_type'] = 'Unknown'
    summary_df.loc[summary_df['Jurisdiction'].str.contains('County|COUNTY', case=False, na=False), 
                   'jurisdiction_type'] = 'Unincorporated'
    summary_df.loc[~summary_df['Jurisdiction'].str.contains('County|COUNTY', case=False, na=False) & 
                   summary_df['Jurisdiction'].notna(), 
                   'jurisdiction_type'] = 'Incorporated'
    
    print(f"\nTotal properties analyzed: {len(summary_df)}")
    print(f"Properties with zoning info: {summary_df['has_zoning'].sum()}")
    print(f"Properties without zoning: {(~summary_df['has_zoning']).sum()}")
    
    print("\n--- Jurisdiction Type Summary ---")
    print(summary_df['jurisdiction_type'].value_counts())
    
    print("\n--- First 10 Properties with Complete Information ---")
    display_summary = summary_df[summary_df['has_zoning']][
        ['address', 'city', 'price', 'lot_sqft', 'existing_adu', 
         'Description', 'jurisdiction_type']
    ].head(10)
    
    # Format price for better display
    display_summary['price'] = display_summary['price'].apply(lambda x: f"${x:,.0f}")
    display_summary['lot_sqft'] = display_summary['lot_sqft'].apply(lambda x: f"{x:,.0f}")
    
    print(display_summary.to_string(index=False))
else:
    print("No zoning information available in the results")


COMPLETE ZONING SUMMARY TABLE

Total properties analyzed: 501
Properties with zoning info: 501
Properties without zoning: 0

--- Jurisdiction Type Summary ---
jurisdiction_type
Incorporated    501
Name: count, dtype: int64

--- First 10 Properties with Complete Information ---
            address        city      price lot_sqft  existing_adu                                            Description jurisdiction_type
15227 Graystone Ave     Norwalk   $869,000    5,068             1                              Single-Family Residential      Incorporated
   1303 N Merona St     Anaheim   $898,000    7,039             1                                                   RS-2      Incorporated
  22849 Leadwell St  West Hills   $830,000    7,499             0                                            RESIDENTIAL      Incorporated
   11373 Rampart Dr      Dublin $1,590,000   13,328             0                                                    R-1      Incorporated
     5841 Arroyo Dr Los An

In [29]:
summary_df

,address,city,price,sqft,lot_sqft,lot_sqft_ratio,existing_adu,County,Jurisdiction,Code,Description,New_Class,has_zoning,jurisdiction_type
0,15227 Graystone Ave,Norwalk,869000,1145.0,5068.0,4.426201,1,LAN,Norwalk,R-1,Single-Family Re...,Suburban residen...,True,Incorporated
1,1303 N Merona St,Anaheim,898000,1278.0,7039.0,5.507825,1,ORA,Anaheim,RS-2,RS-2,Suburban residen...,True,Incorporated
2,22849 Leadwell St,West Hills,830000,1299.0,7499.0,5.772902,0,LAN,Los Angeles,RS-1,RESIDENTIAL,Suburban residen...,True,Incorporated
3,11373 Rampart Dr,Dublin,1590000,2262.0,13328.0,5.892131,0,ALA,Dublin,R-1,R-1,Suburban residen...,True,Incorporated
4,5841 Arroyo Dr,Los Angeles,1148000,1260.0,5564.0,4.415873,0,LAN,Los Angeles,RD2-1,RESIDENTIAL,Multi-family,True,Incorporated
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,3109 Robinette Ave,Baldwin Park,855000,1252.0,5125.0,4.093450,1,LAN,Baldwin Park,R1,R1,Suburban residen...,True,Incorporated
496,24905 Skyland Rd,Los Gatos,1050000,1648.0,27181.0,16.493325,0,SCR,SCR,RA,Residential Agri...,Rural residential,True,Incorporated
497,14621 Chalet Ln,Huntington Beach,1439000,1890.0,6000.0,3.174603,0,ORA,Huntington Beach,RL,Residential Low ...,Suburban residen...,True,Incorporated
498,1925 Vallecito Dr,San Pedro,1280000,1304.0,7321.0,5.614264,0,LAN,Los Angeles,R1-1XL,RESIDENTIAL,Urban residential,True,Incorporated


# 针对设计的时候：先确定能不能后院 （后院尺寸 / 坡度 / setback） - 不能的话你必须只能车库改！



# 目的：计划建设的ADU主要用于长期出租、短期出租（Airbnb），还是家庭成员居住？这会影响设计要求和许可流程。
# 计算的时候，总要算tenant improvement来算（长租短租不影响！）